In [77]:
import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
import os, os.path
import sys
sys.path.insert(0,'../processing/')
sys.path.insert(0,'../')
from datasets import sythtextprovider
from deployment import model_deploy
from nets import txtbox_300
#from processing import image_processing
from image_processing2 import *
from processing import ssd_vgg_preprocessing
import tf_utils
import time
slim = tf.contrib.slim

In [78]:
data_dir = '/Users/xiaodiu/Documents/github/projecttextbox/TextBoxes-TensorFlow/data/sythtext/'
file_name = data_dir + '1.tfrecord'
## test if file_name exists  

#example = tf.python_io.tf_record_iterator(file_name).next()
reader =  tf.python_io.tf_record_iterator(file_name)
#image_buffer, label, bboxes, name= parse_example(example)

In [85]:
def next_batch(reader,net,anchors,batch_size = 32):
    images = []
    b_glocal = []
    b_gsocres = []
    len = 0
    while(len < batch_size):
        try:
            example = reader.next()
            image_buffer, label, bboxes, name= parse_example(example)
            image = tf.image.decode_jpeg(image_buffer,channels=3)
            image, glabels, gbboxes,num =\
                ssd_vgg_preprocessing.preprocess_image(image,label,bboxes, 
                                                (300,300),is_training=True,
                                                data_format='NHWC')
            #glocalisations, gscores = \
            #net.bboxes_encode( gbboxes, anchors,num)
            #b_glocal.append(glocalisations)
            #b_gsocres.append(gscores)
            images.append(image)
            len +=1
        except StopIteration:
            print 'here is end ',i   
    return images, b_glocal, b_gsocres

In [66]:
with tf.Session() as sess:

33